In [ ]:

from flask import Flask

app = Flask(__name__)

@app.route('/dssf/')
def hello():
    return 'Hello, World! this application runing on 192.168.0.105'

if __name__ == '__main__':
    app.run(host='192.168.0.105')

 * Serving Flask app '__main__'
 * Debug mode: off


Cannot assign requested address
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 750, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 99] Cannot assign requested address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-2cf8caedff52>", line 10, in <cell line: 9>
    app.run(host='192.168.0.105')
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving

TypeError: object of type 'NoneType' has no len()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import cv2
import json

app = Flask(__name__)

def uniform_sampling(video, target_frames=30):
    len_frames = video.shape[0]
    interval = int(np.ceil(len_frames / target_frames))
    sampled_video = [video[i] for i in range(0, len_frames, interval)]
    num_pad = target_frames - len(sampled_video)
    if num_pad > 0:
        padding = [video[i] if i >= 0 else video[0] for i in range(-num_pad, 0)]
        sampled_video += padding
    return np.array(sampled_video)

# Load the TensorFlow model
model = tf.saved_model.load('/content/drive/MyDrive/model')  # تأكد من المسار الصحيح للنموذج

# Function to preprocess frames
def preprocess_frames(frames):
    frames = np.array(frames)
    frames = uniform_sampling(frames, target_frames=30)  # Assuming uniform_sampling function is defined
    frames_diff = np.diff(frames, axis=0)
    frames_diff = np.concatenate([frames_diff, np.zeros((1, frames_diff.shape[1], frames_diff.shape[2], frames_diff.shape[3]))], axis=0)
    frames = frames.astype(np.float32) / 255.0
    frames_diff = frames_diff.astype(np.float32) / 255.0
    return frames, frames_diff

# Route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    try:
        print("jkkkkkkkkkk0")
        data = request.get_json()
        frames = data['frames']

        frames, frames_diff = preprocess_frames(frames)

        frames_input = frames[:30]
        frames_diff_input = frames_diff[:30]

        frames_input = tf.convert_to_tensor(frames_input[np.newaxis, ...])
        frames_diff_input = tf.convert_to_tensor(frames_diff_input[np.newaxis, ...])

        prediction = model([frames_input, frames_diff_input], training=False)
        prediction = np.squeeze(prediction)

        is_violent = prediction >= 0.50
        print(is_violent)
        print("jkkkkkkkkkk")

        return jsonify({'is_violent': bool(is_violent)})
    except Exception as e:
        print("jkj")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    print("Starting Flask server...")
    app.run(  )
    print("Flask server started.")

Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Flask server started.


In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

def uniform_sampling(video, target_frames=30):
     # get total frames of input video and calculate sampling interval
    len_frames = video.shape[0]
    print(len_frames)
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and
    sampled_video = []
    for i in range(0, len_frames, interval):
        sampled_video.append(video[i])
    # calculate numer of padded frames and fix it
    num_pad = target_frames - len(sampled_video)
    padding = []
    if num_pad > 0:
        for i in range(-num_pad, 0):
            try:
                padding.append(video[i])
            except:
                padding.append(video[0])
        sampled_video += padding
    # get sampled video
    return np.array(sampled_video)

def Video2Npy(file_path, resize=320, target_frames=30, normalize=False, background_suppress=False):
    cap = cv2.VideoCapture(file_path)
    len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    try:
        for i in range(len_frames):
            _, frame = cap.read()
            frame = cv2.resize(frame, (resize, resize), interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    except Exception as e:
        print("Error: ", file_path, len_frames)
        print(e)
    finally:
        frames = np.array(frames)
        cap.release()
    print(frames.shape)
    frames = uniform_sampling(frames, target_frames=target_frames)

    if background_suppress:
        avg_frame = np.mean(frames, axis=0, dtype=np.uint8)
        frames = np.abs(frames - avg_frame)

    if normalize:
        frames = frames.astype(np.float32) / 255.0
    print(frames.shape)

    return frames

# Function to load the model
def verify_model_loading(model_path):
    model = tf.saved_model.load(model_path)
    print(f"Model loaded from {model_path}")
    return model

# Function to make predictions using the model
def make_prediction(model, frames):
    frames_input = frames[:30].astype(np.float32)  # Take first 30 frames
    frames_diff_input = np.diff(frames, axis=0).astype(np.float32)  # Compute differences

    frames_input = tf.convert_to_tensor(frames_input[np.newaxis, ...])
    frames_diff_input = tf.convert_to_tensor(frames_diff_input[np.newaxis, ...])

    prediction = model([frames_input, frames_diff_input], training=False)
    prediction = np.squeeze(prediction)

    is_violent = prediction >= 0.50
    return is_violent, prediction



# Main function to test on one video
def main():
    video_path = '/content/drive/MyDrive/test3.avi'
    model_path = '/content/drive/MyDrive/model'
    target_frames = 30
    resize = 224
    normalize = True
    background_suppress = True

    # Ensure paths exist
    assert os.path.exists(video_path), f"Video file does not exist: {video_path}"
    assert os.path.exists(model_path), f"Model file does not exist: {model_path}"

    # Load video and preprocess frames
    frames = Video2Npy(video_path, resize=resize, target_frames=target_frames, normalize=normalize, background_suppress=background_suppress)

    print(frames.shape)

    # Load and verify the model
    model = verify_model_loading(model_path)

    # Make a prediction
    is_violent, prediction = make_prediction(model, frames)

    # Print the prediction
    print(f"Prediction: {'Violence' if is_violent else 'Non-Violence'}, Confidence: {prediction:.2f}")

if __name__ == "__main__":
    main()

(150, 224, 224, 3)
150
(30, 224, 224, 3)
(30, 224, 224, 3)
Model loaded from /content/drive/MyDrive/model
Prediction: Violence, Confidence: 0.88


#api code

In [ ]:
!pip install fastapi
!pip install fastapi uvicorn
!pip install colabcode
!pip install "uvicorn[standard]"
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from types import FrameType
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

def uniform_sampling(video, target_frames=30):
         # get total frames of input video and calculate sampling interval
    len_frames = video.shape[0]
    print(len_frames)
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and
    sampled_video = []
    for i in range(0, len_frames, interval):
        sampled_video.append(video[i])
    # calculate numer of padded frames and fix it
    num_pad = target_frames - len(sampled_video)
    padding = []
    if num_pad > 0:
        for i in range(-num_pad, 0):
            try:
                padding.append(video[i])
            except:
                padding.append(video[0])
        sampled_video += padding
    # get sampled video
    return np.array(sampled_video)
# Function to load the model

def verify_model_loading(model_path):
    model = tf.saved_model.load(model_path)
    print(f"Model loaded from {model_path}")
    return model

def Video2Npy(file_path, resize=320, target_frames=30, normalize=False, background_suppress=False):
    cap = cv2.VideoCapture(file_path)
    len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    try:
        for i in range(len_frames):
            _, frame = cap.read()
            frame = cv2.resize(frame, (resize, resize), interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    except Exception as e:
        print("Error: ", file_path, len_frames)
        print(e)
    finally:
        frames = np.array(frames)
        cap.release()
    print(frames.shape)
    frames = uniform_sampling(frames, target_frames=target_frames)

    if background_suppress:
        avg_frame = np.mean(frames, axis=0, dtype=np.uint8)
        frames = np.abs(frames - avg_frame)

    if normalize:
        frames = frames.astype(np.float32) / 255.0
    print(frames.shape)

    return frames
# Function to make predictions using the model
def make_prediction(model, frames):
    frames_input = frames[:30].astype(np.float32)  # Take first 30 frames
    frames_diff_input = np.diff(frames, axis=0).astype(np.float32)  # Compute differences

    frames_input = tf.convert_to_tensor(frames_input[np.newaxis, ...])
    frames_diff_input = tf.convert_to_tensor(frames_diff_input[np.newaxis, ...])

    prediction = model([frames_input, frames_diff_input], training=False)
    prediction = np.squeeze(prediction)

    is_violent = prediction >= 0.50
    return is_violent, prediction

In [ ]:
# !ngrok config add-authtoken 2hvHk6VkJaqPrZRzOsv2griinyY_7JC1k73XaAgMJAFzLXLUC
! ngrok config add-authtoken 2i45sJc1zS7L3kU0d6yb0wfmPAE_27rCfiAQGkSPutoUhBUeq

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# import uvicorn
from typing import Union
from colabcode import ColabCode
from google.colab import drive
import shutil
from pyngrok import ngrok
from fastapi import FastAPI, BackgroundTasks, File, UploadFile
from fastapi.responses import StreamingResponse
from typing import List

In [ ]:
cc = ColabCode(port=8000, code=False)
app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

In [ ]:
# Define paths and constants

TARGET_FRAMES = 30
RESIZE = 224
NORMALIZE = True
BACKGROUND_SUPPRESS = True




In [ ]:
@app.on_event("startup")

# Function to load the model
def model_loading():
    global model
    model_path = '/content/drive/MyDrive/convlstm-model'  # Adjust the path to your TensorFlow model
    tf.saved_model.load(model_path)
    print(f"Model loaded from {model_path}")

<ipython-input-8-090bad9c9258>:1: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
model_path = '/content/drive/MyDrive/convlstm-model'
target_frames = 30
resize = 224
normalize = True
background_suppress = True
@app.post("/")
async def upload_video(video: UploadFile = File(...)):
      try:
        video_path = f"/content/drive/MyDrive/{video.filename}"
        with open(video_path, "wb") as f:
          shutil.copyfileobj(video.file, f)
        frames = Video2Npy(video_path, resize=resize, target_frames=target_frames, normalize=normalize, background_suppress=background_suppress)
        model = verify_model_loading(model_path)
        is_violent, prediction = make_prediction(model, frames)

        print(f"Prediction: {'Violence' if is_violent else 'Non-Violence'}, Confidence: {prediction:.2f}")

        return {"prediction": "Violence" if is_violent else "Non-Violence"}

      except Exception as e:
         return {"prediction": "error", "exception":str(e)}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [516]
INFO:uvicorn.error:Started server process [516]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.


Public URL: https://7b77-34-83-43-48.ngrok-free.app


INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Model loaded from /content/drive/MyDrive/convlstm-model
(144, 224, 224, 3)
144
(30, 224, 224, 3)
Model loaded from /content/drive/MyDrive/convlstm-model
Prediction: Violence, Confidence: 1.00
INFO:     41.233.164.38:0 - "POST / HTTP/1.1" 200 OK
INFO:     54.196.134.84:0 - "GET / HTTP/1.1" 405 Method Not Allowed
